# Tutorial 4: Algoritmos de aprendizaje supervisado clásicos

En este tutorial, describemos los principios de distintos algoritmos clásicos de aprendizaje supervisado: 
- K Vecinos más cercanos
- Naïve Bayes
- Árbol de decisión
- Ensemble: Random Forest, AdaBoost

Ilustramos su uso con una tarea de análisis de sentimientos con el dataset IMDB.

## 0. Preparación del dataset

In [1]:
import spacy

nlp = spacy.load("en")

In [2]:
import pandas as pd

df = pd.read_csv("datos/imdb.csv")
df.shape

(50000, 2)

In [3]:
from pandasql import sqldf

q="""SELECT sentiment, count(*) FROM df GROUP BY sentiment ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,sentiment,count(*)
0,positive,25000
1,negative,25000


In [4]:
q="""SELECT * FROM df WHERE sentiment = "positive";"""
df_positive=sqldf(q)

df_positive = df_positive.sample(n=1000)

q="""SELECT * FROM df WHERE sentiment = "negative";"""
df_negative=sqldf(q)

df_negative = df_negative.sample(n=1000)

In [5]:
df = pd.concat([df_positive, df_negative], ignore_index=True)
df.shape

(2000, 2)

In [6]:
def feature_extraction(text):
    
    mytokens = nlp(text)

    #Guardamos las palabras como características si corresponden a ciertas categorias gramaticales
    mytokens = [ word for word in mytokens if word.pos_ in ["NOUN", "ADJ", "VERB"] ]
    
    #Transformamos las palabras en minusculas
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]

    # return preprocessed list of tokens
    return mytokens

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vector = TfidfVectorizer(tokenizer = feature_extraction, min_df=0., max_df=1.0)

In [8]:
from sklearn.model_selection import train_test_split

X = df['review'] 
ylabels = df['sentiment']

In [9]:
ylabels[:5]

0    positive
1    positive
2    positive
3    positive
4    positive
Name: sentiment, dtype: object

In [10]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
ylabels_encoded = lb.fit_transform(ylabels)

In [11]:
ylabels_encoded[:5]

array([1, 1, 1, 1, 1])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, ylabels_encoded, test_size=0.5)

## 1. K vecinos más cercanos

### 1.1 Ejemplo

In [13]:
from sklearn import neighbors
from sklearn.pipeline import Pipeline

n_neighbors = 15

knn = neighbors.KNeighborsClassifier(n_neighbors)

model_knn = Pipeline([('vectorizing', tfidf_vector),
                 ('learning', knn)])

In [14]:
import time

start = time.time()

model_knn.fit(X_train, y_train)

end = time.time()
print("tiempo para entrenar:" + str(end - start))

tiempo para entrenar:58.915780544281006


In [15]:
from sklearn import metrics

start = time.time()

predicted = model_knn.predict(X_test)

end = time.time()
print("tiempo para predecir:" + str(end - start))

tiempo para predecir:56.45247983932495


In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

confusion_matrix = confusion_matrix(y_test, predicted)

print(confusion_matrix)

print(classification_report(y_test, predicted))

[[384 124]
 [139 353]]
              precision    recall  f1-score   support

           0       0.73      0.76      0.74       508
           1       0.74      0.72      0.73       492

    accuracy                           0.74      1000
   macro avg       0.74      0.74      0.74      1000
weighted avg       0.74      0.74      0.74      1000



### 1.2 Descripción

<img src="img/knn.png" />


Ventajas:

- Simple: número de vecinos, distancia entre vectores (Euclidiana, Manhattan)
- No requiere entrenamiento (_lazy_)
- Interesante para resolver problemas de clasificación de datos no linealmente separables

Desventajas:

- En la fase de predicciones, el rendimiento baja si el dataset de entrenamiento es grande. No hay generalización, se necesita calcular distancias con cada instancia.
- Si las caracteristicas utilizan escalas distintas, impactar mucho el calculo de la distancia (se requiere normalizar)
- Requiere memoria


## 2. Naïve Bayes

### 2.1 Ejemplo

In [17]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

model_nb = Pipeline([('vectorizing', tfidf_vector),
                 ('learning', nb)])

In [18]:
import time

start = time.time()

model_nb.fit(X_train, y_train)

end = time.time()
print("tiempo para entrenar:" + str(end - start))

tiempo para entrenar:54.50274157524109


In [ ]:
import time

start = time.time()

predicted = model_nb.predict(X_test)

end = time.time()
print("tiempo para predecir:" + str(end - start))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test, predicted)

print(confusion_matrix)

print(classification_report(y_test, predicted))

In [ ]:

def get_salient_words(nb_clf, vect, class_ind):
    """Return salient words for given class
    Parameters
    ----------
    nb_clf : a Naive Bayes classifier (e.g. MultinomialNB, BernoulliNB)
    vect : CountVectorizer
    class_ind : int
    Returns
    -------
    list
        a sorted list of (word, log prob) sorted by log probability in descending order.
    """

    words = vect.get_feature_names()
    zipped = list(zip(words, nb_clf.feature_log_prob_[class_ind]))
    sorted_zip = sorted(zipped, key=lambda t: t[1], reverse=True)

    return sorted_zip

neg_salient_top = get_salient_words(nb, tfidf_vector, 0)[:10]
pos_salient_top = get_salient_words(nb, tfidf_vector, 1)[:10]

print(neg_salient_top)
print(pos_salient_top)

### 2.2 Descripción

Teorema de Bayes:

<img src="img/naivebayes.jpeg" />

**Modelo probabilístico que asume que existe una independencia entre las caractéristicas.**


¿Qué se aprende?: probabilidades


Ventajas:

- Cuando la suposición independiente se mantiene, entonces este clasificador da una precisión excepcional.
- Es fácil de implementar ya que sólo se debe calcular la probabilidad
- Funciona bien con dimensiones altas como la clasificación de texto.

Desventajas:

- Si la suposición independiente no se mantiene, entonces el rendimiento es muy bajo.

- Suavizar resulta ser un paso obligado cuando la probabilidad de una característica resulta ser cero en una clase.

## 3. Árbol de decisión

### 3.1 Ejemplo

In [ ]:
from sklearn import tree

my_tree = tree.DecisionTreeClassifier()

model_tree = Pipeline([('vectorizing', tfidf_vector),
                 ('learning', my_tree)])

In [ ]:
import time

start = time.time()

model_tree.fit(X_train, y_train)

end = time.time()
print("tiempo para entrenar:" + str(end - start))

In [ ]:
import time

start = time.time()

predicted = model_tree.predict(X_test)

end = time.time()
print("tiempo para predecir:" + str(end - start))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test, predicted)

print(confusion_matrix)

print(classification_report(y_test, predicted))

In [ ]:
#!pip install graphviz

In [ ]:
from graphviz import Source
graph = Source( tree.export_graphviz(my_tree, out_file=None, feature_names=tfidf_vector.get_feature_names()))
graph.format = 'png'
graph.render('tree_render',view=True)

In [ ]:
txt_representation = tree.export_text(my_tree, feature_names=tfidf_vector.get_feature_names(), max_depth=10)
print(txt_representation)

### 3.2 Descripción

¿Qué se aprende?: un arbol de preguntas para dividir los datos de entrenamiento.
    
<img src="img/CART.png"></img> 

En nuestro ejemplo, el nodo 1 está totalmente desenredado (label: "Grape"). El nodo 2 tiene dos labeles, entonces preguntamos otra pregunta:

<img src="img/CART2.png"></img>

Para construir un árbol eficiente, el punto importante es identificar qué preguntas formular y cuándo. Por lo tanto, necesitamos <b>cuantificar</b> en qué medida una pregunta permite desenredar los labeles. Para hacer eso se utiliza 2 métricas:
- el coeficiente de '<b>Gini impurity</b>': mide que tan desenredados están los labeles de un nodo.
- el coeficiente de '<b>Information Gain</b>': mide cuánto una pregunta permite bajar el 'Gini impurity'.

<img src="img/CART3.png"></img>

Utilizaremos estas métricas para estimar qué preguntas hacer.

<img src="img/CART4.png"></img>

Para saber qué preguntas formular, cada nodo itera sobre las características de los datos a su disposición y define una lista de preguntas posibles.

Una vez una pregunta elegida, se divide los datos en dos según la respuesta a la pregunta.

<img src="img/CART5.png"></img>

El coeficiente de Gini impurity representa la probabilidad de ser incorrecto si asigna aleatoriamente una etiqueta a un ejemplo del mismo conjunto. Por ejemplo, en los dos ejemplos siguientes: ¿Cuál es la probabilidad de equivocarse si asignamos una etiqueta del recipiente B a un dato del recipiente A?

Ejemplo 1:
<img src="img/CART-6.png"></img>

Ejemplo 2:
<img src="img/CART-7.png"></img>


La métrica de **Information Gain** permite medir qué pregunta optimiza el coeficiente de Gini impurity.

Por cada nodo, empezamos por medir el coeficiente de Gini impurity de los labeles disponibles. Luego, por cada pregunta calculamos el coeficiente de Gini impurity de los dos sub-conjuntos de datos obtenidos.

<img src="img/CART-8.png"></img>

Calculamos la incerteza (<i>impurity</i>) promedia ponderada para los dos subconjuntos de datos obtenidos. Por ejemplo:

<img src="img/CART-9.png"></img>

Finalmente, conservamos la pregunta que permite optimizar la ganancia de información (Information Gain). En nuestro ejemplo:

<b>Information Gain = 0.64 - 0.50 = 0.14</b>


Ventajas:

- es interpretable e intuitivo
- no requiere normalizar o re-escalar los datos
- los datos incompletos no impactan el entrenamiento


Desventajas:

- pequeños cambios en el dataset de entrenamiento puede generar grandes impactos en la estructura del árbol
- overfitting (modelo demasiado ajustado al dataset de training)
- costoso en computación en la fase de entrenamiento


## 4. Ensemble Learning

### 3.1 Ejemplo

- Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=50, max_depth=None, min_samples_split=10, random_state=0)

model_random_forest = Pipeline([('vectorizing', tfidf_vector),
                 ('learning', random_forest)])

In [ ]:
import time

start = time.time()

model_random_forest.fit(X_train, y_train)

end = time.time()
print("tiempo para entrenar:" + str(end - start))


In [ ]:
import time

start = time.time()

predicted = model_random_forest.predict(X_test)

end = time.time()
print("tiempo para predecir:" + str(end - start))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test, predicted)

print(confusion_matrix)

print(classification_report(y_test, predicted))

- Ada Boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators=100)

model_ada = Pipeline([('vectorizing', tfidf_vector),
                 ('learning', ada)])

In [ ]:
model_ada.fit(X_train, y_train)

In [ ]:
predicted = model_ada.predict(X_test)

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test, predicted)

print(confusion_matrix)

print(classification_report(y_test, predicted))

### 3.2 Descripción

Combinar varios clasificadores debiles:

<img src="img/ensemble.png" />


Funcionamiento detallado de RandomForest: https://www.youtube.com/watch?v=J4Wdy0Wc_xQ (20 minutos)

Funcionamento detallado de AdaBoost: https://www.youtube.com/watch?v=LsK-xG1cLYA (20 minutos)


Ventajas:

- Reduce el overfitting de los arboles de decisión
